In [1]:
from evaluate_model import evaluate_agent
import d3rlpy
from d3rlpy import dataset
from d3rlpy.algos.bcq import BCQ
import numpy as np
import csv

from wget import download
from env.abr import ABRSimEnv
import matplotlib.pyplot as plt

import ray
import ray.rllib.agents.ppo as ppo
from train_baseline import myEnv

from policies import BBAAgent
import pandas as pd

In [2]:
old_cql_model = d3rlpy.algos.DiscreteCQL.from_json("d3rlpy_logs/DiscreteCQL_20211115220329/params.json")
OLD_CQL_MODEL_WEIGHTS = "d3rlpy_logs/DiscreteCQL_20211115220329/model_8950.pt"
old_cql_model.load_model(OLD_CQL_MODEL_WEIGHTS)

2021-12-07 16:41.33 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteCQL generated_maxlen=100000 real_ratio=1.0


In [3]:
cql_alpha_0_model = d3rlpy.algos.DiscreteCQL.from_json("d3rlpy_logs/DiscreteCQL_20211206161713/params.json")
CQL_MODEL_ALPHA_0_WEIGHTS = "d3rlpy_logs/DiscreteCQL_20211206161713/model_1432000.pt"
cql_alpha_0_model.load_model(CQL_MODEL_ALPHA_0_WEIGHTS)

2021-12-07 16:41.34 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteCQL generated_maxlen=100000 real_ratio=1.0


In [4]:
cql_alpha_1_model = d3rlpy.algos.DiscreteCQL.from_json("d3rlpy_logs/DiscreteCQL_20211207035334/params.json")
CQL_MODEL_ALPHA_1_WEIGHTS = "d3rlpy_logs/DiscreteCQL_20211207035334/model_1432000.pt"
cql_alpha_1_model.load_model(CQL_MODEL_ALPHA_1_WEIGHTS)

2021-12-07 16:41.35 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteCQL generated_maxlen=100000 real_ratio=1.0


In [5]:
cql_alpha_01_model = d3rlpy.algos.DiscreteCQL.from_json("d3rlpy_logs/DiscreteCQL_20211206221540/params.json")
CQL_MODEL_ALPHA_01_WEIGHTS = "d3rlpy_logs/DiscreteCQL_20211206221540/model_1432000.pt"
cql_alpha_01_model.load_model(CQL_MODEL_ALPHA_01_WEIGHTS)

2021-12-07 16:41.36 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteCQL generated_maxlen=100000 real_ratio=1.0


In [6]:
cql_alpha_10_model = d3rlpy.algos.DiscreteCQL.from_json("d3rlpy_logs/DiscreteCQL_20211207092707/params.json")
CQL_MODEL_ALPHA_10_WEIGHTS = "d3rlpy_logs/DiscreteCQL_20211207092707/model_1432000.pt"
cql_alpha_10_model.load_model(CQL_MODEL_ALPHA_10_WEIGHTS)

2021-12-07 16:41.37 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteCQL generated_maxlen=100000 real_ratio=1.0


In [7]:
old_bcq_model = d3rlpy.algos.DiscreteBCQ.from_json("d3rlpy_logs/DiscreteBCQ_20211115222009/params.json")
OLD_BCQ_MODEL_WEIGHTS = "d3rlpy_logs/DiscreteBCQ_20211115222009/model_8950.pt"
old_bcq_model.load_model(OLD_BCQ_MODEL_WEIGHTS)

2021-12-07 16:41.38 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteBCQ generated_maxlen=100000 real_ratio=1.0


In [8]:
bcq_model = d3rlpy.algos.DiscreteBCQ.from_json("d3rlpy_logs/DiscreteBCQ_20211206161607/params.json")
BCQ_MODEL_WEIGHTS = "d3rlpy_logs/DiscreteBCQ_20211206161607/model_1432000.pt"
bcq_model.load_model(BCQ_MODEL_WEIGHTS)

2021-12-07 16:41.39 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteBCQ generated_maxlen=100000 real_ratio=1.0


In [9]:
ray.init()
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["lr"] = 1e-2
config["lambda"] = 0.96
config["gamma"] = 0.96
config["entropy_coeff_schedule"] = [(0, 0.2), (2500*490, 0.0)]
config["model"]["fcnet_hiddens"] = [64, 32]
config["model"]["fcnet_activation"] = "relu" 
config["rollout_fragment_length"] = 490

ppo_model = ppo.PPOTrainer(config=config, env=myEnv)
ppo_model.load_checkpoint("models/ppo/checkpoint_005201/checkpoint-5201")

2021-12-07 16:41:44,146	INFO trainer.py:722 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2021-12-07 16:41:44,148	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=1 num_envs_per_worker=1 rollout_fragment_length=490)! Auto-adjusting `rollout_fragment_length` to 4000.
2021-12-07 16:41:44,149	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-12-07 16:41:44,150	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=58156) Setting up environment..


(RolloutWorker pid=58156) 2021-12-07 16:42:17,436	WARNING deprecation.py:45 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` instead. This will raise an error in the future!
2021-12-07 16:42:18,571	WARNING deprecation.py:45 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` instead. This will raise an error in the future!
2021-12-07 16:42:19,525	INFO trainable.py:124 -- Trainable.setup took 35.383 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [10]:
bba_model = BBAAgent(env = ABRSimEnv())

In [11]:
models = [old_cql_model, cql_alpha_0_model, cql_alpha_01_model, cql_alpha_1_model, cql_alpha_10_model, old_bcq_model, bcq_model, ppo_model, bba_model]

In [12]:
old_cql_eval = evaluate_agent(old_cql_model)

Setting up environment..


In [13]:
cql_alpha_0_eval = evaluate_agent(cql_alpha_0_model)

Setting up environment..


In [14]:
cql_alpha_01_eval = evaluate_agent(cql_alpha_01_model)

Setting up environment..


In [15]:
cql_alpha_1_eval = evaluate_agent(cql_alpha_1_model)

Setting up environment..


In [17]:
cql_alpha_10_eval = evaluate_agent(cql_alpha_10_model)

Setting up environment..


In [16]:
old_bcq_eval = evaluate_agent(old_bcq_model)

Setting up environment..


In [18]:
bcq_eval = evaluate_agent(bcq_model)

Setting up environment..


In [20]:
ppo_eval = evaluate_agent(ppo_model, ppo=True)

Setting up environment..


2021-12-07 16:57:17,307	WARNING deprecation.py:45 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


In [19]:
bba_eval = evaluate_agent(bba_model, bba=True)

Setting up environment..


In [21]:
evals = [old_cql_eval, cql_alpha_0_eval, cql_alpha_01_eval, cql_alpha_1_eval, cql_alpha_10_eval, bcq_eval, old_bcq_eval, bcq_eval, ppo_eval, bba_eval]

In [22]:
def eval_to_df(eval):
    cols = ["traj"] + list(eval[0].keys())
    iters = len(eval[0]['action'])
    rows = []
    for ind, traj in enumerate(eval):
        for iter in range(iters):
            row = [ind]
            row.append(traj['action'][iter])
            row.append(traj['buffer_length'][iter])
            row.append(traj['action_bitrate'][iter])
            row.append(traj['download_time'][iter])
            row.append(traj['throughput'][iter])
            row.append(traj['reward'][iter])
            rows.append(row)
    
    return pd.DataFrame(rows, columns=cols)

In [23]:
old_cql_df = eval_to_df(old_cql_eval)
cql_a0_df = eval_to_df(cql_alpha_0_eval)
cql_a01_df = eval_to_df(cql_alpha_01_eval)
cql_a1_df = eval_to_df(cql_alpha_1_eval)
cql_a10_df = eval_to_df(cql_alpha_10_eval)
old_bcq_df = eval_to_df(old_bcq_eval)
bcq_df = eval_to_df(bcq_eval)
ppo_df = eval_to_df(ppo_eval)
bba_df = eval_to_df(bba_eval)
dfs = [old_cql_df, cql_a0_df, cql_a01_df, cql_a1_df, cql_a10_df, bcq_df, old_bcq_df, bcq_df, ppo_df, bba_df]

In [24]:
import plotly.express as px
import plotly.graph_objects as go

In [25]:
summary_dict = {"cql" : [cql_a1_df.groupby(by="traj").mean(), cql_a1_df.groupby(by="traj").sum()],
       # "cql (a=0)" : [cql_a0_df.groupby(by="traj").mean(), cql_a0_df.groupby(by="traj").sum()],
       # "cql (a=.1)" : [cql_a01_df.groupby(by="traj").mean(), cql_a01_df.groupby(by="traj").sum()],
       # "cql (a=10)" : [cql_a10_df.groupby(by="traj").mean(), cql_a10_df.groupby(by="traj").sum()],
       # "old cql" : [old_cql_df.groupby(by="traj").mean(), old_cql_df.groupby(by="traj").sum()],
        "bcq": [bcq_df.groupby(by="traj").mean(), bcq_df.groupby(by="traj").sum()],
       # "old bcq": [bcq_beta_df.groupby(by="traj").mean(), bcq_beta_df.groupby(by="traj").sum()],
        "ppo" : [ppo_df.groupby(by="traj").mean(), ppo_df.groupby(by="traj").sum()],
        "bba" : [bba_df.groupby(by="traj").mean(), bba_df.groupby(by="traj").sum()],
       # "cql (alpha=0)" : [cql_a0_df.groupby(by="traj").mean(), cql_a0_df.groupby(by="traj").sum()],
       # "cql (alpha=10)" : [cql_a10_df.groupby(by="traj").mean(), cql_a10_df.groupby(by="traj").sum()]
}

In [26]:
cols = ["model", "traj", "average throughput", "total reward"]
rows = []
for m in summary_dict:
    avg_thpt = list(summary_dict[m][0]["throughput"])
    tot_rew = list(summary_dict[m][1]["reward"])
    for ind, tup in enumerate(zip(avg_thpt, tot_rew)):
        thpt, rew = tup
        rows.append([m, ind, thpt, rew])

summary_df = pd.DataFrame(rows, columns=cols)

In [27]:
px.scatter(summary_df, x="average throughput", y="total reward", color="model", opacity=0.4, trendline="ols",
        title="Model evaluations over 200 trajectories")

In [28]:
import plotly.figure_factory as ff

In [29]:
hist_data = []
group_labels = []
for model in summary_dict:
    hist_data.append(summary_dict[model][1]["reward"])
    group_labels.append(model)

fig = ff.create_distplot(hist_data, group_labels, bin_size=100, show_hist=False)
fig.show()

In [30]:
fig = px.ecdf(summary_df, x="total reward", color="model")
fig.show()

In [41]:
summary_dict2 = {"cql" : [cql_a1_df.groupby(by="traj").mean(), cql_a1_df.groupby(by="traj").sum()],
       # "cql (a=0)" : [cql_a0_df.groupby(by="traj").mean(), cql_a0_df.groupby(by="traj").sum()],
       "cql (a=.1)" : [cql_a01_df.groupby(by="traj").mean(), cql_a01_df.groupby(by="traj").sum()],
       "cql (a=10)" : [cql_a10_df.groupby(by="traj").mean(), cql_a10_df.groupby(by="traj").sum()],
       "old cql" : [old_cql_df.groupby(by="traj").mean(), old_cql_df.groupby(by="traj").sum()],
       "bcq": [bcq_df.groupby(by="traj").mean(), bcq_df.groupby(by="traj").sum()],
       "old bcq": [old_bcq_df.groupby(by="traj").mean(), old_bcq_df.groupby(by="traj").sum()],
       # "ppo" : [ppo_df.groupby(by="traj").mean(), ppo_df.groupby(by="traj").sum()],
       # "bba" : [bba_df.groupby(by="traj").mean(), bba_df.groupby(by="traj").sum()],
       # "cql (alpha=0)" : [cql_a0_df.groupby(by="traj").mean(), cql_a0_df.groupby(by="traj").sum()],
       # "cql (alpha=10)" : [cql_a10_df.groupby(by="traj").mean(), cql_a10_df.groupby(by="traj").sum()]
}

In [42]:
cols2 = ["model", "traj", "average throughput", "total reward"]
rows2 = []
for m in summary_dict2:
    avg_thpt = list(summary_dict2[m][0]["throughput"])
    tot_rew = list(summary_dict2[m][1]["reward"])
    for ind, tup in enumerate(zip(avg_thpt, tot_rew)):
        thpt, rew = tup
        rows2.append([m, ind, thpt, rew])

summary_df2 = pd.DataFrame(rows2, columns=cols2)

In [43]:
px.scatter(summary_df2, x="average throughput", y="total reward", color="model", opacity=0.4, trendline="ols",
        title="Model evaluations over 200 trajectories")

In [44]:
hist_data2 = []
group_labels2 = []
for model in summary_dict2:
    hist_data2.append(summary_dict2[model][1]["reward"])
    group_labels2.append(model)

fig = ff.create_distplot(hist_data2, group_labels2, bin_size=100, show_hist=False)
fig.show()

In [45]:
fig = px.ecdf(summary_df2, x="total reward", color="model")
fig.show()